# Тестовое задание PostgreSQL

У вас SQL база с таблицами:  
1) users(userid, age)
2) purchases (purchaseid, userid, itemid, date)
3) items (itemid, price).

Будет здорово, если ваше решение оформлено в виде работающего кода и основательно протестировано на придуманных данных.  
Для тестирования можно использовать онлайн-редактор https://sqliteonline.com/  
Диалект - PostgreSQL 

Напишите SQL запросы для расчета следующих метрик:
А) какую сумму в среднем в месяц тратит:  
- пользователи в возрастном диапазоне от 18 до 25 лет включительно  
- пользователи в возрастном диапазоне от 26 до 35 лет включительно  
Б) в каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая  
В) какой товар обеспечивает дает наибольший вклад в выручку за последний год  
Г) топ-3 товаров по выручке и их доля в общей выручке за любой год  

<hr>
Создаём таблицы

In [ ]:
CREATE TABLE IF NOT EXISTS users (
    userid SERIAL PRIMARY KEY, 
    age INTEGER
    );

CREATE TABLE IF NOT EXISTS items (
    itemid SERIAL PRIMARY KEY,
    price INTEGER
    );

CREATE TABLE IF NOT EXISTS purchases (
    purchaseid SERIAL PRIMARY KEY,
    userid INTEGER REFERENCES Users(userid),
    itemid INTEGER REFERENCES Items(itemid),
    date TIMESTAMP NOT NULL
    );

<hr>
Добавляем записи в БД

In [ ]:
INSERT INTO users(age)
VALUES
    (18),
    (25),
    (26),
    (30),
    (35),
    (36);

INSERT INTO items(price)
VALUES
    (1),
    (3),
    (5),
    (7);

INSERT INTO purchases(userid, itemid, date)
VALUES
    (3, 1, '2022-11-01'),
    (6, 2, '2022-11-03'),
    (5, 4, '2022-11-01'),
    (2, 3, '2022-11-19'),
    (6, 4, '2022-12-01'),
    (5, 2, '2022-12-01'),
    (2, 3, '2022-12-01'),
    (2, 4, '2022-12-02'),
    (1, 2, '2022-12-05'),
    (2, 4, '2022-12-07'),
    (5, 2, '2022-12-09'),
    (2, 4, '2022-12-15'),
    (1, 1, '2022-12-17'),
    (4, 2, '2022-12-21'),
    (6, 2, '2022-12-23'),
    (2, 2, '2023-01-01'),
    (6, 3, '2023-01-02'),
    (6, 4, '2023-01-02'),
    (3, 4, '2023-01-11'),
    (6, 2, '2023-01-13'),
    (5, 1, '2022-01-25'),
    (4, 3, '2023-01-29');

<hr>

А) какую сумму в среднем в месяц тратит:  
- пользователи в возрастном диапазоне от 18 до 25 лет включительно  
- пользователи в возрастном диапазоне от 26 до 35 лет включительно  

In [ ]:
SELECT
    DATE_TRUNC('month', pt.date) AS year_month,
    ur.age_range,
    ROUND(SUM(it.price)::DECIMAL / COUNT(DISTINCT pt.userid), 2) AS mean_sum
FROM purchases AS pt

LEFT JOIN items AS it
    ON it.itemid=pt.itemid
    
LEFT JOIN (
    SELECT
        userid,
        CASE
            when age BETWEEN 18 AND 25 THEN '18-25'
            when age BETWEEN 26 AND 35 THEN '26-35'
        END AS age_range
    FROM users
) AS ur
    ON pt.userid=ur.userid

WHERE age_range IS NOT NULL
GROUP BY year_month, age_range
ORDER BY year_month, age_range

<hr>

Б) в каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая

In [ ]:
SELECT
    DATE_TRUNC('month', pt.date) AS month_range,
    SUM(it.price) AS revenue
FROM purchases AS pt

LEFT JOIN items AS it
    ON it.itemid=pt.itemid

LEFT JOIN (SELECT userid, age FROM users) AS ut
    ON pt.userid=ut.userid

WHERE age > 35
GROUP BY month_range
ORDER BY revenue DESC
LIMIT 1

<hr>
В) какой товар обеспечивает дает наибольший вклад в выручку за последний год

In [ ]:
WITH purchases_table AS (SELECT * FROM purchases)

SELECT
    pt.itemid,
    DATE_TRUNC('year', pt.date) AS year_range,
    SUM(it.price) AS price
FROM purchases_table AS pt

LEFT JOIN items AS it
	ON it.itemid=pt.itemid

WHERE DATE_PART('year', pt.date) = (SELECT MAX(DATE_PART('year', date)) AS date FROM purchases_table)
GROUP BY year_range, pt.itemid
ORDER BY price DESC
LIMIT 1

<hr>
Г) топ-3 товаров по выручке и их доля в общей выручке за любой год

In [ ]:
WITH revenue_table AS (
    SELECT
        pt.itemid,
        DATE_TRUNC('year', pt.date) AS year_range,
        it.price AS price
    FROM purchases AS pt

    LEFT JOIN items AS it
        ON it.itemid=pt.itemid
)

SELECT
    itemid,
    min(year_range) AS year,
    SUM(price) AS revenue
FROM revenue_table

WHERE year_range < DATE '2023-01-01'
GROUP BY itemid
ORDER BY revenue DESC
LIMIT 3